In [72]:
import numpy as np
import pandas as pd
import os
import re

In [73]:
def interpolate_values(latitude, data_pd):
    
    data_pd = data_pd.sort_values(by=data_pd.columns[0], ascending=True)
    
    rad_ext_values = []
    for i in range(1, 13):
          
        index = np.where(data_pd.iloc[:,0]<=latitude)[0][-1]    
        interp_func = np.interp(latitude,np.array(data_pd.iloc[index:index+2,0]),
                                np.array(data_pd.iloc[index:index+2,i]))
        rad_ext_values.append(round(interp_func,13))
    
    dict_values = {int(data_pd.columns[1+i]) : rad_ext_values[i] for i in range(0,12)}
    return dict_values

In [74]:
def eto_hargreaves(tas_data_path, kt, latitude, radiacion_mon_data):
    
    radiacion_extraterrestre_df = pd.read_csv(radiacion_mon_data)
    radiacion_inter = interpolate_values(latitude, radiacion_extraterrestre_df)
    
    tas_data = pd.read_csv(tas_data_path)
    tas_data.iloc[:,0] = pd.to_datetime(tas_data.iloc[:,0])
    tas_data = tas_data.set_index('date')

    tas_max = tas_data['tasmax']
    tas_min = tas_data['tasmin']
      
    day_month = {1:31,2:28,3:31,4:30,5:31,6:30,
                 7:31,8:31,9:30,10:31,11:30,12:31}
 
    substracion_tas = tas_max - tas_min
    addition_tas = tas_max + tas_min
    
    eto_mon = np.ones(len(tas_data)) 
    for im in range(1,13):
              
        index = tas_max.index.month == im   
        sub_tas = substracion_tas[index]
        add_tas = addition_tas[index]
        factor = round(0.0135*kt*day_month[im],6)
        eto_mon[index] = factor*radiacion_inter[im]*((sub_tas.values)**0.5) * (add_tas.values/2 +17.78)
        
    eto_hargreaves = pd.DataFrame({'date':tas_data.index})    
    eto_hargreaves['eto'] = eto_mon 
    
    
    eto_aux = eto_mon.reshape((int(len(tas_data)/12),12)) 
    month = ['Aug','Sep', 'Oct', 'Nov', 'Dic', 'Jan', 'Feb',
             'Mar', 'Apr', 'May', 'Jun', 'Jul']
    start_year, end_year = tas_data.index.year[0], tas_data.index.year[-1]
    years = [f'{i}-{i+1}' for i in range(start_year, end_year)]
    eto_hargreaves_hydr_year = pd.DataFrame(data=eto_aux,columns=month, index=years) 
    
    eto_hargreaves = round(eto_hargreaves,1)
    eto_hargreaves_hydr_year = round(eto_hargreaves_hydr_year,1)    

    eto_hargreaves = eto_hargreaves.set_index('date')
    return eto_hargreaves, eto_hargreaves_hydr_year
    

In [75]:
def eto_thornthwaite(tas_data_path, latitude, factor_correc_data):
    
    factor_por_horas_sol_df = pd.read_csv(factor_correc_data)
    factor_sol_inter = interpolate_values(latitude, factor_por_horas_sol_df)
    
    tas_data = pd.read_csv(tas_data_path)
    tas_data.iloc[:,0] = pd.to_datetime(tas_data.iloc[:,0])
    tas_data = tas_data.set_index('date')
    
    tas_avg = tas_data.iloc[:,:2].mean(axis=1)

    day_month = {1:31,2:28,3:31,4:30,5:31,6:30,
                 7:31,8:31,9:30,10:31,11:30,12:31}
 
    aux_mon = np.ones(len(tas_data))
    
    for im in range(1,13):
        index = tas_data.index.month == im   
        aux_mon[index] = np.array(tas_avg[index]/5)**1.514
    
    aux_mon = np.array(aux_mon.flatten())

    data_hidro_year = aux_mon.reshape((int(len(tas_avg)/12),12))
    data_hidro_year = pd.DataFrame(data_hidro_year)
    
    I = data_hidro_year.sum(axis=1)
    a = (6.75*(1E-7)*(I**3))-(0.771*(1E-4)*(I**2))+(0.01792*I)+0.49239
    
    I_s = np.repeat(I,12)
    a_s = np.repeat(a,12)
    
    eto_mon = 16*((10*(np.array(tas_avg)/I_s))**a_s)

    
    eto_mon_corrected = np.ones(len(tas_data))
    
    for im in range(1,13):
        index = tas_data.index.month == im   
        eto_mon_corrected[index] = eto_mon[index]*factor_sol_inter[im]
    
    eto_thornthwaite = pd.DataFrame({'date':tas_data.index})    
    eto_thornthwaite['eto'] = eto_mon_corrected
    
    eto_aux = eto_mon_corrected.reshape((int(len(tas_avg)/12),12)) 
    month = ['Aug','Sep', 'Oct', 'Nov', 'Dic', 'Jan', 'Feb',
             'Mar', 'Apr', 'May', 'Jun', 'Jul']
    start_year, end_year = tas_data.index.year[0], tas_data.index.year[-1]
    years = [f'{i}-{i+1}' for i in range(start_year, end_year)]
    eto_thornthwaite_hydr_year = pd.DataFrame(data=eto_aux,columns=month, index=years)
    
    eto_thornthwaite = eto_thornthwaite.set_index('date')
    eto_thornthwaite = round(eto_thornthwaite,1)
    eto_thornthwaite_hydr_year = round(eto_thornthwaite_hydr_year,1)
    return eto_thornthwaite, eto_thornthwaite_hydr_year
    

In [76]:
def output_folder_directory(var, outputh_folder_path):
    # Create output folte if it does not exist
    output_path = os.path.join(outputh_folder_path, var)
    isExist = os.path.exists(output_path)

    if isExist == False:
        os.makedirs(output_path) 
    
    os.chdir(output_path)
    return output_path

In [77]:
def eto_hargreaves_thornthwaite(tas_data_path, latitude, kt, rad_extr_path, horas_sol_path, scenario_type, output_path):

    serie_eto_hargreaves, hydro_yr_eto_hargreaves = eto_hargreaves(tas_data_path, kt, latitude, rad_extr_path)
    serie_eto_thornthwaite, hydro_yr_eto_thornthwaite = eto_thornthwaite(tas_data_path, latitude, horas_sol_path)   
    
    eto_series = pd.concat([serie_eto_hargreaves,serie_eto_thornthwaite], axis=1)
    eto_series.columns = ['eto_hargreaves_mm','eto_thornthwaite_mm']
    
    output_folder_directory(scenario_type, output_path)
    outfilename = tas_data_path.split("\\")
    eto_series.to_csv(outfilename[-1].replace('tasmaxmin','eto'))
    print(outfilename[-1].replace('tasmaxmin','eto'))
    return eto_series

### Convert from delta_scenario eto

In [102]:
kt = 0.162
latitude = 16.56888889
scenario_type = 'Median'

main_path = r"D:\PROYECTOS\03_Represa_Sapoco\05_CC\03_cc_delta_scenarios\eto"
files     = os.listdir(main_path)
dir_files = [os.path.join(main_path, file) for file in files]

rad_extr_path  = r"D:\PROYECTOS\03_Represa_Sapoco\05_CC\cc_eto\radiacion_extraterrestre.csv"
horas_sol_path = r"D:\PROYECTOS\03_Represa_Sapoco\05_CC\cc_eto\factor_correccion_duracion_media_horas_sol.csv"
output_path    = r"D:\PROYECTOS\03_Represa_Sapoco\05_CC\04_cc_eto"

for tas_data_path in dir_files:
    eto_hargreaves_thornthwaite(tas_data_path, latitude, kt, rad_extr_path, horas_sol_path, scenario_type, output_path)

Median_San_Ignacio_de_Velasco_eto_ssp245_2036_2065.csv
Median_San_Ignacio_de_Velasco_eto_ssp585_2036_2065.csv


### FAO

In [117]:
eto_folder = r"D:\PROYECTOS\03_Represa_Sapoco\05_CC\04_cc_eto\Median"
os.chdir(eto_folder)

files = os.listdir()
print(files)
eto_fao_folder = r"D:\PROYECTOS\03_Represa_Sapoco\05_CC\04_cc_eto\fao\export_data"
output_folder  = rf"D:\PROYECTOS\03_Represa_Sapoco\05_CC\04_cc_eto\{scenario_type}"

for file in files:
    input_data = os.path.join(eto_folder, file)
    print(input_data)
    data_eto = pd.read_csv(input_data)
    date = pd.to_datetime(data_eto.date)

    date = pd.date_range(f'{date[0].year}-08-31', f'{date[len(date)-1].year}-07-31', freq='M')    
    data_eto['date'] = pd.to_datetime(date)
    #os.remove(file)
    
    filename = file.replace('eto','tasmaxmin')[:-4]
    input_data   = os.path.join(eto_fao_folder, f'{filename}.ETo')
    eto_fao_data = pd.read_csv(input_data) 
    eto_fao = list(eto_fao_data.index[6:])
    
    eto_fao = [float(re.findall("\d+\.\d+", i)[0]) for i in eto_fao]   
    
    day = date.day
    print(len(eto_fao))
    data_eto['FAO_Penman-Monteith_mm'] = eto_fao*day
    
    data_eto = data_eto.set_index('date')
    
    data_eto['promedio_eto_mm'] = data_eto.mean(axis=1)
    
    
    os.chdir(output_folder)
    print(file)
    data_eto.to_csv(file)
    
    
    

['Median_San_Ignacio_de_Velasco_eto_ssp245_2036_2065.csv', 'Median_San_Ignacio_de_Velasco_eto_ssp585_2036_2065.csv']
D:\PROYECTOS\03_Represa_Sapoco\05_CC\04_cc_eto\Median\Median_San_Ignacio_de_Velasco_eto_ssp245_2036_2065.csv
180
Median_San_Ignacio_de_Velasco_eto_ssp245_2036_2065.csv
D:\PROYECTOS\03_Represa_Sapoco\05_CC\04_cc_eto\Median\Median_San_Ignacio_de_Velasco_eto_ssp585_2036_2065.csv
180
Median_San_Ignacio_de_Velasco_eto_ssp585_2036_2065.csv
